# Working with Data

In [1]:
%matplotlib inline

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [7]:
## Download the data from open datasets
training_data = datasets.FashionMNIST(root = "data", train = True, download = True, transform = ToTensor(),)
test_data = datasets.FashionMNIST(root = "data", train = False, download = True, transform = ToTensor(),)

In [30]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data,batch_size = batch_size)

for X, y in test_dataloader:
    print(X)
    print("shape of X[N,C,H,W]:", X.shape)
    print("shpae of y:", y.shape, y.dtype)
    break

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.

# Creating Models

In [19]:
#get cpu of gpu device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print("using {} device".format(device))

using cpu device


In [21]:
#Define model
class FashionMNIST(nn.Module):
    def __init__(self):
        super(FashionMNIST,self).__init__()
        self.flatten = nn.Flatten()
        self. linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )
        
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = FashionMNIST().to(device)
print(model)

FashionMNIST(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [22]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= 1e-3)

In [35]:
#predictions on the training dataset, backpropagation, optimizing
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        #Compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)
        
        #Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch %100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [46]:
#in test data
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%,Avg loss: {test_loss:>8f} \n")

In [47]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------------
loss: 0.888285 [    0/60000]
loss: 0.975268 [ 6400/60000]
loss: 0.766145 [12800/60000]
loss: 0.906015 [19200/60000]
loss: 0.814490 [25600/60000]
loss: 0.778966 [32000/60000]
loss: 0.859775 [38400/60000]
loss: 0.859101 [44800/60000]
loss: 0.816302 [51200/60000]
loss: 0.913648 [57600/60000]
Test Error: 
 Accuracy: 75.5%,Avg loss: 0.856086 

Epoch 2
--------------------------
loss: 0.870258 [    0/60000]
loss: 0.959838 [ 6400/60000]
loss: 0.751350 [12800/60000]
loss: 0.893972 [19200/60000]
loss: 0.803917 [25600/60000]
loss: 0.767906 [32000/60000]
loss: 0.847459 [38400/60000]
loss: 0.850872 [44800/60000]
loss: 0.804223 [51200/60000]
loss: 0.904205 [57600/60000]
Test Error: 
 Accuracy: 75.9%,Avg loss: 0.843804 

Epoch 3
--------------------------
loss: 0.853750 [    0/60000]
loss: 0.945213 [ 6400/60000]
loss: 0.738242 [12800/60000]
loss: 0.882989 [19200/60000]
loss: 0.794531 [25600/60000]
loss: 0.757745 [32000/60000]
loss: 0.836115 [38400/60000]
loss: 0.84

In [48]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [50]:
model = FashionMNIST()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [51]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
